### Banking (Java)

Consider following Java program with a class `Account` that contains `balance` as the sole field. Classes `Deposit` and `Withdraw` allow to create threads that will deposit to and withdraw from a given `Account` by incrementing or decrementing its `balance` field. They do so by depositing or withdrawing successively increasing amounts from `1` to `M`. The main program creates in total `N` threads, half `Deposit` and half `Withdraw` threads.

In [1]:
%%writefile Bank.java
class Account {
   int balance = 0;
}

class Deposit extends Thread {
    Account a;
    public Deposit(Account a) {
        this.a = a;
    }
    public void run() {
        for (int i = 1; i <= Bank.M; i++) a.balance += i;
    }
}

class Withdraw extends Thread {
    Account a;
    public Withdraw(Account a) {
        this.a = a;
    }
    public void run() {
        for (int i = 1; i <= Bank.M; i++) a.balance -= i;
    }
}

public class Bank {
    static int M = 1000; // maximum amount deposited or withdrawn
    static int N = 1000; // total number of Deposit, Withdraw threads
    public static void main(String args[]) {
        Account a = new Account();
        Thread w[] = new Thread[N];
        for (int i = 0; i < N; i++) {
            // create half Deposit, half Withdraw threads
            if (i % 2 == 0) w[i] = new Deposit(a);
            else w[i] = new Withdraw(a);
            w[i].start();
        }
        for (int i = 0; i < N; i++) {
            // wait for all threads to terminate
            try {w[i].join();
            } catch (Exception e) {}
        }
        System.out.println(a.balance);
    }
}

Overwriting Bank.java


In [2]:
!javac Bank.java

In [3]:
!java Bank

1279565


Since in total the same amount is deposited as withdrawn, the balance at the end should be zero. Run it with varying values of `M` and `N`. Sometimes the final balance will be zero and sometimes not. Explain why! As a reminder, reading or writing an integer variable is atomic in Java. [2 points]

The main issue with the above class is there is no synchronization mechanism that ensures that despositing and withdrawing operations are done atomically meaning multiple threads can overlap and cause unpredictable behavior. Another reason for this could be race conditions. If 2 threads try to update the value of a.balance concurrently the value being read may not have been updated which can lead to inconsistent behavior. 

Java has methods for atomically updating integer variables, see https://docs.oracle.com/javase/tutorial/essential/concurrency/atomicvars.html and https://docs.oracle.com/en/java/javase/19/docs/api/java.base/java/util/concurrent/atomic/AtomicInteger.html. Copy above code to the cell bellow and modify it to use an `AtomicInteger` instead of `int`. Test if you now get zero as the final balance! [6 points]

In [4]:
%%writefile Bank.java
import java.util.concurrent.atomic.AtomicInteger;

class Account {
   AtomicInteger balance = new AtomicInteger(0);
}

class Deposit extends Thread {
    Account a;
    public Deposit(Account a) {
        this.a = a;
    }
    public void run() {
        for (int i = 1; i <= Bank.M; i++) a.balance.incrementAndGet();
    }
}

class Withdraw extends Thread {
    Account a;
    public Withdraw(Account a) {
        this.a = a;
    }
    public void run() {
        for (int i = 1; i <= Bank.M; i++) a.balance.decrementAndGet();
    }
}

public class Bank {
    static int M = 500; // maximum amount deposited or withdrawn
    static int N = 1000; // total number of Deposit, Withdraw threads
    public static void main(String args[]) {
        Account a = new Account();
        Thread w[] = new Thread[N];
        for (int i = 0; i < N; i++) {
            // create half Deposit, half Withdraw threads
            if (i % 2 == 0) w[i] = new Deposit(a);
            else w[i] = new Withdraw(a);
            w[i].start();
        }
        for (int i = 0; i < N; i++) {
            // wait for all threads to terminate
            try {w[i].join();
            } catch (Exception e) {}
        }
        System.out.println(a.balance);
    }
}

Overwriting Bank.java


In [5]:
!javac Bank.java

In [6]:
!java Bank

0
